# Key-Value Stores: Redis & DynamoDB Patterns

Key-value stores are the simplest form of NoSQL databases, offering lightning-fast reads and writes by mapping unique keys to values. This notebook covers fundamental concepts, patterns, and practical implementations.

## 1. The Key-Value Model

### What is a Key-Value Store?

A key-value store is a data storage paradigm where:
- **Key**: A unique identifier (string, number, or composite)
- **Value**: Any data - strings, JSON, binary blobs, or complex objects

```
┌─────────────────────────────────────────────────────┐
│                  KEY-VALUE STORE                    │
├─────────────────────┬───────────────────────────────┤
│        KEY          │           VALUE               │
├─────────────────────┼───────────────────────────────┤
│  "user:1001"        │  {"name": "Alice", "age": 30} │
│  "session:abc123"   │  {"user_id": 1001, "ttl": 3600}│
│  "cache:products"   │  [...product list...]         │
│  "counter:views"    │  42857                        │
└─────────────────────┴───────────────────────────────┘
```

### Core Operations

| Operation | Description | Time Complexity |
|-----------|-------------|------------------|
| GET | Retrieve value by key | O(1) |
| SET | Store key-value pair | O(1) |
| DELETE | Remove key-value pair | O(1) |
| EXISTS | Check if key exists | O(1) |
| TTL | Set expiration time | O(1) |

## 2. Common Use Cases

### 🚀 Caching
- Database query results
- API responses
- Computed values
- Static content

### 🔐 Session Management
- User authentication tokens
- Shopping cart data
- Temporary user state

### 📊 Real-time Analytics
- Page view counters
- Rate limiting
- Leaderboards

### 📬 Message Queues
- Task queues
- Pub/Sub messaging
- Event streaming

## 3. Python Key-Value Simulation

Let's build a simple key-value store simulator to understand the core concepts.

In [ ]:
import time
import json
from datetime import datetime, timedelta
from typing import Any, Optional, Dict, List
from collections import OrderedDict


class SimpleKeyValueStore:
    """A simple in-memory key-value store with TTL support."""
    
    def __init__(self):
        self._store: Dict[str, Any] = {}
        self._expiry: Dict[str, float] = {}  # Key -> expiry timestamp
    
    def set(self, key: str, value: Any, ttl_seconds: Optional[int] = None) -> bool:
        """Store a key-value pair with optional TTL."""
        self._store[key] = value
        if ttl_seconds:
            self._expiry[key] = time.time() + ttl_seconds
        elif key in self._expiry:
            del self._expiry[key]
        return True
    
    def get(self, key: str) -> Optional[Any]:
        """Retrieve value by key, respecting TTL."""
        if self._is_expired(key):
            self.delete(key)
            return None
        return self._store.get(key)
    
    def delete(self, key: str) -> bool:
        """Remove a key-value pair."""
        if key in self._store:
            del self._store[key]
            self._expiry.pop(key, None)
            return True
        return False
    
    def exists(self, key: str) -> bool:
        """Check if key exists and is not expired."""
        if self._is_expired(key):
            self.delete(key)
            return False
        return key in self._store
    
    def ttl(self, key: str) -> int:
        """Get remaining TTL in seconds. Returns -1 if no TTL, -2 if key doesn't exist."""
        if key not in self._store:
            return -2
        if key not in self._expiry:
            return -1
        remaining = self._expiry[key] - time.time()
        return max(0, int(remaining))
    
    def _is_expired(self, key: str) -> bool:
        """Check if a key has expired."""
        if key in self._expiry:
            return time.time() > self._expiry[key]
        return False
    
    def keys(self, pattern: str = "*") -> List[str]:
        """Get all keys matching pattern (* for all)."""
        # Clean expired keys first
        for key in list(self._store.keys()):
            if self._is_expired(key):
                self.delete(key)
        
        if pattern == "*":
            return list(self._store.keys())
        # Simple prefix matching
        prefix = pattern.rstrip("*")
        return [k for k in self._store.keys() if k.startswith(prefix)]


# Demo usage
kv = SimpleKeyValueStore()

# Basic operations
kv.set("user:1001", {"name": "Alice", "email": "alice@example.com"})
kv.set("user:1002", {"name": "Bob", "email": "bob@example.com"})
kv.set("session:xyz", {"user_id": 1001, "role": "admin"}, ttl_seconds=3600)

print("=== Basic Operations ===")
print(f"GET user:1001 -> {kv.get('user:1001')}")
print(f"EXISTS user:1001 -> {kv.exists('user:1001')}")
print(f"TTL session:xyz -> {kv.ttl('session:xyz')} seconds")
print(f"KEYS user:* -> {kv.keys('user:*')}")

## 4. Caching Patterns

Let's implement common caching patterns used with key-value stores.

In [ ]:
import hashlib
import functools


class CachePatterns:
    """Common caching patterns implementation."""
    
    def __init__(self, store: SimpleKeyValueStore):
        self.store = store
        self.stats = {"hits": 0, "misses": 0}
    
    # Pattern 1: Cache-Aside (Lazy Loading)
    def cache_aside(self, key: str, fetch_fn, ttl: int = 300):
        """
        Cache-Aside Pattern:
        1. Check cache first
        2. On miss, fetch from source and populate cache
        """
        cached = self.store.get(key)
        if cached is not None:
            self.stats["hits"] += 1
            return cached
        
        self.stats["misses"] += 1
        value = fetch_fn()
        self.store.set(key, value, ttl)
        return value
    
    # Pattern 2: Write-Through
    def write_through(self, key: str, value: Any, write_fn, ttl: int = 300):
        """
        Write-Through Pattern:
        1. Write to cache
        2. Write to database synchronously
        """
        self.store.set(key, value, ttl)
        write_fn(key, value)  # Write to DB
        return value
    
    # Pattern 3: Write-Behind (Write-Back)
    def __init_write_behind(self):
        self._write_queue = []
    
    def write_behind(self, key: str, value: Any, ttl: int = 300):
        """
        Write-Behind Pattern:
        1. Write to cache immediately
        2. Queue database write for async processing
        """
        self.store.set(key, value, ttl)
        if not hasattr(self, '_write_queue'):
            self._write_queue = []
        self._write_queue.append((key, value))
        return value
    
    def get_hit_ratio(self) -> float:
        """Calculate cache hit ratio."""
        total = self.stats["hits"] + self.stats["misses"]
        return self.stats["hits"] / total if total > 0 else 0.0


# Simulate database
fake_database = {
    "product:1": {"name": "Laptop", "price": 999.99},
    "product:2": {"name": "Mouse", "price": 29.99},
    "product:3": {"name": "Keyboard", "price": 79.99},
}

def fetch_from_db(product_id: str):
    """Simulates slow database fetch."""
    print(f"  [DB] Fetching {product_id} from database...")
    time.sleep(0.1)  # Simulate latency
    return fake_database.get(product_id)


# Demo Cache-Aside pattern
cache = CachePatterns(SimpleKeyValueStore())

print("=== Cache-Aside Pattern Demo ===")
print("\nFirst request (cache miss):")
result1 = cache.cache_aside("product:1", lambda: fetch_from_db("product:1"))
print(f"  Result: {result1}")

print("\nSecond request (cache hit):")
result2 = cache.cache_aside("product:1", lambda: fetch_from_db("product:1"))
print(f"  Result: {result2}")

print(f"\nCache Stats: {cache.stats}")
print(f"Hit Ratio: {cache.get_hit_ratio():.1%}")

## 5. Session Management Pattern

In [ ]:
import uuid
import secrets


class SessionStore:
    """Session management using key-value store."""
    
    SESSION_TTL = 3600  # 1 hour
    SESSION_PREFIX = "session:"
    
    def __init__(self, store: SimpleKeyValueStore):
        self.store = store
    
    def create_session(self, user_id: int, metadata: dict = None) -> str:
        """Create a new session for a user."""
        session_id = secrets.token_urlsafe(32)
        session_key = f"{self.SESSION_PREFIX}{session_id}"
        
        session_data = {
            "user_id": user_id,
            "created_at": datetime.now().isoformat(),
            "last_accessed": datetime.now().isoformat(),
            "metadata": metadata or {}
        }
        
        self.store.set(session_key, session_data, self.SESSION_TTL)
        return session_id
    
    def get_session(self, session_id: str) -> Optional[dict]:
        """Retrieve and refresh session."""
        session_key = f"{self.SESSION_PREFIX}{session_id}"
        session = self.store.get(session_key)
        
        if session:
            # Refresh session TTL on access
            session["last_accessed"] = datetime.now().isoformat()
            self.store.set(session_key, session, self.SESSION_TTL)
        
        return session
    
    def invalidate_session(self, session_id: str) -> bool:
        """Invalidate/logout a session."""
        session_key = f"{self.SESSION_PREFIX}{session_id}"
        return self.store.delete(session_key)
    
    def update_session(self, session_id: str, data: dict) -> bool:
        """Update session metadata."""
        session = self.get_session(session_id)
        if session:
            session["metadata"].update(data)
            session_key = f"{self.SESSION_PREFIX}{session_id}"
            self.store.set(session_key, session, self.SESSION_TTL)
            return True
        return False


# Demo session management
session_store = SessionStore(SimpleKeyValueStore())

print("=== Session Management Demo ===")

# Create session on login
session_id = session_store.create_session(
    user_id=1001,
    metadata={"ip": "192.168.1.1", "user_agent": "Chrome/120"}
)
print(f"\nCreated session: {session_id[:20]}...")

# Retrieve session
session = session_store.get_session(session_id)
print(f"Session data: {json.dumps(session, indent=2)}")

# Add to shopping cart
session_store.update_session(session_id, {"cart": [{"product_id": 1, "qty": 2}]})
updated_session = session_store.get_session(session_id)
print(f"\nUpdated session cart: {updated_session['metadata'].get('cart')}")

## 6. Redis Data Structures

Redis extends the basic key-value model with rich data structures. Let's simulate them.

```
┌────────────────────────────────────────────────────────────────┐
│                    REDIS DATA STRUCTURES                       │
├───────────────┬────────────────────────────────────────────────┤
│   Strings     │  Simple values, counters, binary data          │
│   Lists       │  Ordered collections, queues, stacks           │
│   Sets        │  Unique unordered elements, tags               │
│   Hashes      │  Field-value pairs, object storage             │
│   Sorted Sets │  Ranked elements, leaderboards                 │
└───────────────┴────────────────────────────────────────────────┘
```

In [ ]:
class RedisSimulator:
    """Simulates Redis data structures and operations."""
    
    def __init__(self):
        self._store: Dict[str, Any] = {}
    
    # ============ STRING OPERATIONS ============
    def set(self, key: str, value: str) -> str:
        """SET key value"""
        self._store[key] = value
        return "OK"
    
    def get(self, key: str) -> Optional[str]:
        """GET key"""
        return self._store.get(key)
    
    def incr(self, key: str) -> int:
        """INCR key - Increment integer value"""
        val = int(self._store.get(key, 0))
        self._store[key] = str(val + 1)
        return val + 1
    
    def incrby(self, key: str, amount: int) -> int:
        """INCRBY key amount"""
        val = int(self._store.get(key, 0))
        self._store[key] = str(val + amount)
        return val + amount
    
    def append(self, key: str, value: str) -> int:
        """APPEND key value"""
        current = self._store.get(key, "")
        self._store[key] = current + value
        return len(self._store[key])
    
    # ============ LIST OPERATIONS ============
    def lpush(self, key: str, *values) -> int:
        """LPUSH key value [value...] - Push to head"""
        if key not in self._store:
            self._store[key] = []
        for v in values:
            self._store[key].insert(0, v)
        return len(self._store[key])
    
    def rpush(self, key: str, *values) -> int:
        """RPUSH key value [value...] - Push to tail"""
        if key not in self._store:
            self._store[key] = []
        self._store[key].extend(values)
        return len(self._store[key])
    
    def lpop(self, key: str) -> Optional[str]:
        """LPOP key - Pop from head"""
        if key in self._store and self._store[key]:
            return self._store[key].pop(0)
        return None
    
    def rpop(self, key: str) -> Optional[str]:
        """RPOP key - Pop from tail"""
        if key in self._store and self._store[key]:
            return self._store[key].pop()
        return None
    
    def lrange(self, key: str, start: int, stop: int) -> List:
        """LRANGE key start stop - Get range of elements"""
        if key not in self._store:
            return []
        lst = self._store[key]
        if stop == -1:
            stop = len(lst)
        else:
            stop = stop + 1
        return lst[start:stop]
    
    def llen(self, key: str) -> int:
        """LLEN key - List length"""
        return len(self._store.get(key, []))


# Demo String and List operations
redis = RedisSimulator()

print("=== Redis String Operations ===")
redis.set("greeting", "Hello")
print(f"SET greeting 'Hello' -> {redis.get('greeting')}")

redis.append("greeting", ", World!")
print(f"APPEND greeting ', World!' -> {redis.get('greeting')}")

redis.set("counter", "0")
print(f"\nINCR counter: {redis.incr('counter')}")
print(f"INCRBY counter 10: {redis.incrby('counter', 10)}")

print("\n=== Redis List Operations (Queue) ===")
# Use as a message queue
redis.rpush("tasks", "task1", "task2", "task3")
print(f"RPUSH tasks -> {redis.lrange('tasks', 0, -1)}")

print(f"LPOP tasks (dequeue): {redis.lpop('tasks')}")
print(f"Remaining: {redis.lrange('tasks', 0, -1)}")

In [ ]:
class RedisSimulator(RedisSimulator):
    """Extended with Sets and Hashes."""
    
    # ============ SET OPERATIONS ============
    def sadd(self, key: str, *members) -> int:
        """SADD key member [member...]"""
        if key not in self._store:
            self._store[key] = set()
        before = len(self._store[key])
        self._store[key].update(members)
        return len(self._store[key]) - before
    
    def smembers(self, key: str) -> set:
        """SMEMBERS key - Get all members"""
        return self._store.get(key, set())
    
    def sismember(self, key: str, member: str) -> bool:
        """SISMEMBER key member"""
        return member in self._store.get(key, set())
    
    def sinter(self, *keys) -> set:
        """SINTER key [key...] - Intersection"""
        if not keys:
            return set()
        result = self._store.get(keys[0], set()).copy()
        for key in keys[1:]:
            result &= self._store.get(key, set())
        return result
    
    def sunion(self, *keys) -> set:
        """SUNION key [key...] - Union"""
        result = set()
        for key in keys:
            result |= self._store.get(key, set())
        return result
    
    def scard(self, key: str) -> int:
        """SCARD key - Set cardinality"""
        return len(self._store.get(key, set()))
    
    # ============ HASH OPERATIONS ============
    def hset(self, key: str, field: str, value: str) -> int:
        """HSET key field value"""
        if key not in self._store:
            self._store[key] = {}
        is_new = field not in self._store[key]
        self._store[key][field] = value
        return 1 if is_new else 0
    
    def hget(self, key: str, field: str) -> Optional[str]:
        """HGET key field"""
        if key in self._store:
            return self._store[key].get(field)
        return None
    
    def hmset(self, key: str, mapping: dict) -> str:
        """HMSET key field value [field value...]"""
        if key not in self._store:
            self._store[key] = {}
        self._store[key].update(mapping)
        return "OK"
    
    def hgetall(self, key: str) -> dict:
        """HGETALL key"""
        return self._store.get(key, {})
    
    def hincrby(self, key: str, field: str, amount: int) -> int:
        """HINCRBY key field amount"""
        if key not in self._store:
            self._store[key] = {}
        val = int(self._store[key].get(field, 0))
        self._store[key][field] = str(val + amount)
        return val + amount


# Demo Sets
redis = RedisSimulator()

print("=== Redis Set Operations ===")
# User interests/tags
redis.sadd("user:1:interests", "python", "redis", "docker", "kubernetes")
redis.sadd("user:2:interests", "python", "postgresql", "redis", "aws")

print(f"User 1 interests: {redis.smembers('user:1:interests')}")
print(f"User 2 interests: {redis.smembers('user:2:interests')}")

common = redis.sinter("user:1:interests", "user:2:interests")
print(f"Common interests: {common}")

all_topics = redis.sunion("user:1:interests", "user:2:interests")
print(f"All topics: {all_topics}")

print("\n=== Redis Hash Operations ===")
# Store user profile as hash
redis.hmset("user:1001", {
    "name": "Alice",
    "email": "alice@example.com",
    "login_count": "0",
    "balance": "100"
})

print(f"User profile: {redis.hgetall('user:1001')}")
print(f"User name: {redis.hget('user:1001', 'name')}")

# Increment login count
redis.hincrby("user:1001", "login_count", 1)
print(f"After login: {redis.hgetall('user:1001')}")

In [ ]:
class RedisSimulator(RedisSimulator):
    """Extended with Sorted Sets."""
    
    # ============ SORTED SET OPERATIONS ============
    def zadd(self, key: str, mapping: Dict[str, float]) -> int:
        """ZADD key score member [score member...]"""
        if key not in self._store:
            self._store[key] = {}  # member -> score
        added = 0
        for member, score in mapping.items():
            if member not in self._store[key]:
                added += 1
            self._store[key][member] = score
        return added
    
    def zscore(self, key: str, member: str) -> Optional[float]:
        """ZSCORE key member"""
        if key in self._store:
            return self._store[key].get(member)
        return None
    
    def zincrby(self, key: str, amount: float, member: str) -> float:
        """ZINCRBY key increment member"""
        if key not in self._store:
            self._store[key] = {}
        score = self._store[key].get(member, 0) + amount
        self._store[key][member] = score
        return score
    
    def zrank(self, key: str, member: str) -> Optional[int]:
        """ZRANK key member - Get rank (0-based, ascending)"""
        if key not in self._store or member not in self._store[key]:
            return None
        sorted_members = sorted(self._store[key].items(), key=lambda x: x[1])
        for i, (m, _) in enumerate(sorted_members):
            if m == member:
                return i
        return None
    
    def zrevrank(self, key: str, member: str) -> Optional[int]:
        """ZREVRANK key member - Get rank (0-based, descending)"""
        if key not in self._store or member not in self._store[key]:
            return None
        sorted_members = sorted(self._store[key].items(), key=lambda x: -x[1])
        for i, (m, _) in enumerate(sorted_members):
            if m == member:
                return i
        return None
    
    def zrange(self, key: str, start: int, stop: int, withscores: bool = False) -> List:
        """ZRANGE key start stop [WITHSCORES] - Ascending order"""
        if key not in self._store:
            return []
        sorted_items = sorted(self._store[key].items(), key=lambda x: x[1])
        if stop == -1:
            stop = len(sorted_items)
        else:
            stop = stop + 1
        result = sorted_items[start:stop]
        if withscores:
            return result
        return [m for m, _ in result]
    
    def zrevrange(self, key: str, start: int, stop: int, withscores: bool = False) -> List:
        """ZREVRANGE key start stop [WITHSCORES] - Descending order"""
        if key not in self._store:
            return []
        sorted_items = sorted(self._store[key].items(), key=lambda x: -x[1])
        if stop == -1:
            stop = len(sorted_items)
        else:
            stop = stop + 1
        result = sorted_items[start:stop]
        if withscores:
            return result
        return [m for m, _ in result]
    
    def zcard(self, key: str) -> int:
        """ZCARD key - Sorted set cardinality"""
        return len(self._store.get(key, {}))


# Demo Sorted Sets - Leaderboard
redis = RedisSimulator()

print("=== Redis Sorted Set - Game Leaderboard ===")

# Add players with scores
redis.zadd("leaderboard", {
    "player:alice": 2500,
    "player:bob": 3200,
    "player:charlie": 2800,
    "player:diana": 4100,
    "player:eve": 1900
})

print("\nTop 3 Players:")
top3 = redis.zrevrange("leaderboard", 0, 2, withscores=True)
for rank, (player, score) in enumerate(top3, 1):
    print(f"  #{rank}: {player.split(':')[1].capitalize()} - {int(score)} pts")

# Player gains points
print("\nAlice gains 1000 points...")
new_score = redis.zincrby("leaderboard", 1000, "player:alice")
print(f"Alice new score: {int(new_score)}")
print(f"Alice new rank: #{redis.zrevrank('leaderboard', 'player:alice') + 1}")

print("\nUpdated Top 3:")
top3 = redis.zrevrange("leaderboard", 0, 2, withscores=True)
for rank, (player, score) in enumerate(top3, 1):
    print(f"  #{rank}: {player.split(':')[1].capitalize()} - {int(score)} pts")

## 7. DynamoDB Concepts

Amazon DynamoDB is a fully managed, serverless NoSQL database. Unlike Redis, it's designed for:
- **Persistence** over in-memory speed
- **Horizontal scaling** across partitions
- **Strong consistency** options

### Key Concepts

```
┌────────────────────────────────────────────────────────────────────┐
│                       DYNAMODB TABLE                               │
├────────────────────────────────────────────────────────────────────┤
│  Partition Key (PK)  │  Sort Key (SK)    │  Attributes             │
├──────────────────────┼───────────────────┼─────────────────────────┤
│  USER#1001           │  PROFILE          │  name, email, age       │
│  USER#1001           │  ORDER#2024-001   │  total, status, items   │
│  USER#1001           │  ORDER#2024-002   │  total, status, items   │
│  USER#1002           │  PROFILE          │  name, email, age       │
│  PRODUCT#SKU123      │  METADATA         │  name, price, stock     │
└──────────────────────┴───────────────────┴─────────────────────────┘
```

| Concept | Description |
|---------|-------------|
| **Partition Key (PK)** | Primary key for data distribution. Determines which partition stores the item. |
| **Sort Key (SK)** | Optional. Enables range queries within a partition. |
| **GSI (Global Secondary Index)** | Alternate query patterns with different PK/SK. |
| **LSI (Local Secondary Index)** | Same PK, different SK for additional sort orders. |

In [ ]:
from dataclasses import dataclass, field
from typing import Dict, List, Any, Optional, Tuple
import operator


@dataclass
class DynamoDBItem:
    """Represents a DynamoDB item."""
    pk: str
    sk: str
    attributes: Dict[str, Any] = field(default_factory=dict)
    
    def to_dict(self) -> Dict[str, Any]:
        return {"PK": self.pk, "SK": self.sk, **self.attributes}


class DynamoDBSimulator:
    """
    Simulates core DynamoDB operations.
    Uses composite key (PK + SK) for item storage.
    """
    
    def __init__(self, table_name: str):
        self.table_name = table_name
        self._items: Dict[Tuple[str, str], DynamoDBItem] = {}
        self._gsi: Dict[str, Dict[Tuple[str, str], DynamoDBItem]] = {}
    
    def put_item(self, pk: str, sk: str, **attributes) -> dict:
        """
        PutItem - Insert or replace an item.
        """
        item = DynamoDBItem(pk=pk, sk=sk, attributes=attributes)
        self._items[(pk, sk)] = item
        
        # Update GSIs
        for gsi_name, gsi_data in self._gsi.items():
            gsi_pk_attr = gsi_data["pk_attr"]
            gsi_sk_attr = gsi_data.get("sk_attr")
            
            if gsi_pk_attr in attributes:
                gsi_pk = attributes[gsi_pk_attr]
                gsi_sk = attributes.get(gsi_sk_attr, sk) if gsi_sk_attr else sk
                gsi_data["items"][(gsi_pk, gsi_sk)] = item
        
        return {"ConsumedCapacity": {"TableName": self.table_name, "WriteCapacityUnits": 1}}
    
    def get_item(self, pk: str, sk: str) -> Optional[Dict]:
        """
        GetItem - Retrieve a single item by PK + SK.
        """
        item = self._items.get((pk, sk))
        return item.to_dict() if item else None
    
    def query(self, pk: str, sk_begins_with: str = None, sk_between: Tuple[str, str] = None,
              scan_index_forward: bool = True, limit: int = None) -> List[Dict]:
        """
        Query - Retrieve items by PK with optional SK conditions.
        """
        results = []
        
        for (item_pk, item_sk), item in self._items.items():
            if item_pk != pk:
                continue
            
            if sk_begins_with and not item_sk.startswith(sk_begins_with):
                continue
            
            if sk_between:
                start, end = sk_between
                if not (start <= item_sk <= end):
                    continue
            
            results.append(item.to_dict())
        
        # Sort by SK
        results.sort(key=lambda x: x["SK"], reverse=not scan_index_forward)
        
        if limit:
            results = results[:limit]
        
        return results
    
    def delete_item(self, pk: str, sk: str) -> bool:
        """DeleteItem - Remove an item."""
        if (pk, sk) in self._items:
            del self._items[(pk, sk)]
            return True
        return False
    
    def create_gsi(self, gsi_name: str, pk_attr: str, sk_attr: str = None):
        """
        Create a Global Secondary Index.
        """
        self._gsi[gsi_name] = {
            "pk_attr": pk_attr,
            "sk_attr": sk_attr,
            "items": {}
        }
        
        # Backfill existing items
        for item in self._items.values():
            if pk_attr in item.attributes:
                gsi_pk = item.attributes[pk_attr]
                gsi_sk = item.attributes.get(sk_attr, item.sk) if sk_attr else item.sk
                self._gsi[gsi_name]["items"][(gsi_pk, gsi_sk)] = item
    
    def query_gsi(self, gsi_name: str, pk: str, sk_begins_with: str = None) -> List[Dict]:
        """
        Query a Global Secondary Index.
        """
        if gsi_name not in self._gsi:
            raise ValueError(f"GSI {gsi_name} does not exist")
        
        results = []
        for (gsi_pk, gsi_sk), item in self._gsi[gsi_name]["items"].items():
            if gsi_pk != pk:
                continue
            if sk_begins_with and not gsi_sk.startswith(sk_begins_with):
                continue
            results.append(item.to_dict())
        
        results.sort(key=lambda x: x["SK"])
        return results


# Demo DynamoDB Single-Table Design
db = DynamoDBSimulator("UserOrders")

print("=== DynamoDB Single-Table Design ===")

# Store user profile
db.put_item("USER#1001", "PROFILE", 
            name="Alice", email="alice@example.com", tier="premium")

# Store user orders
db.put_item("USER#1001", "ORDER#2024-001",
            total=150.00, status="delivered", items=["laptop_stand"])
db.put_item("USER#1001", "ORDER#2024-002",
            total=89.99, status="shipped", items=["keyboard"])
db.put_item("USER#1001", "ORDER#2024-003",
            total=299.99, status="processing", items=["monitor"])

# Another user
db.put_item("USER#1002", "PROFILE",
            name="Bob", email="bob@example.com", tier="standard")
db.put_item("USER#1002", "ORDER#2024-001",
            total=45.00, status="delivered", items=["mouse"])

print("\n--- GetItem: User Profile ---")
profile = db.get_item("USER#1001", "PROFILE")
print(json.dumps(profile, indent=2))

print("\n--- Query: All Orders for User#1001 ---")
orders = db.query("USER#1001", sk_begins_with="ORDER#")
for order in orders:
    print(f"  {order['SK']}: ${order['total']} - {order['status']}")

In [ ]:
# DynamoDB Access Patterns with GSI

print("=== DynamoDB Global Secondary Index (GSI) ===")

# Create a GSI to query by email
db.create_gsi("EmailIndex", pk_attr="email")

# Create a GSI to query by status
db.create_gsi("StatusIndex", pk_attr="status")

print("\n--- Query GSI: Find user by email ---")
results = db.query_gsi("EmailIndex", "alice@example.com")
print(f"Found: {results[0]['name'] if results else 'Not found'}")

print("\n--- Query GSI: All 'delivered' orders ---")
delivered = db.query_gsi("StatusIndex", "delivered")
for item in delivered:
    print(f"  {item['PK']} - {item['SK']}: ${item['total']}")


# Common DynamoDB Access Patterns
print("\n" + "="*50)
print("DYNAMODB ACCESS PATTERNS REFERENCE")
print("="*50)

patterns = """
┌─────────────────────────────────────────────────────────────────────────┐
│ Access Pattern              │ Key Design                               │
├─────────────────────────────┼──────────────────────────────────────────┤
│ Get user by ID              │ PK=USER#<id>, SK=PROFILE                 │
│ Get user's orders           │ PK=USER#<id>, SK begins_with ORDER#      │
│ Get specific order          │ PK=USER#<id>, SK=ORDER#<order_id>        │
│ Get orders by date range    │ PK=USER#<id>, SK between ORDER#start/end │
│ Find user by email (GSI)    │ GSI PK=email                             │
│ Find orders by status (GSI) │ GSI PK=status, SK=created_at             │
└─────────────────────────────┴──────────────────────────────────────────┘
"""
print(patterns)

## 8. Redis vs DynamoDB Comparison

| Feature | Redis | DynamoDB |
|---------|-------|----------|
| **Storage** | In-memory (with persistence options) | Disk-based (SSD) |
| **Latency** | Sub-millisecond | Single-digit milliseconds |
| **Data Structures** | Rich (strings, lists, sets, hashes, sorted sets, streams) | Items with attributes |
| **Querying** | Key-based only | PK/SK queries, GSI/LSI |
| **Scaling** | Cluster mode sharding | Automatic partitioning |
| **Durability** | RDB/AOF snapshots | Highly durable (3 AZs) |
| **Cost Model** | Memory-based | Provisioned/On-demand capacity |
| **Best For** | Caching, sessions, real-time | Primary database, serverless apps |

### When to Use Which?

**Choose Redis when:**
- Sub-millisecond latency is critical
- Data can be reconstructed from another source
- Using as a cache layer
- Need pub/sub, rate limiting, or leaderboards

**Choose DynamoDB when:**
- Need a persistent, durable database
- Predictable scaling requirements
- Serverless architecture
- Multiple access patterns with GSIs

In [ ]:
# Rate Limiter Pattern (Common Redis Use Case)

class SlidingWindowRateLimiter:
    """
    Rate limiter using sliding window counter.
    Common pattern for API rate limiting.
    """
    
    def __init__(self, store: SimpleKeyValueStore, max_requests: int, window_seconds: int):
        self.store = store
        self.max_requests = max_requests
        self.window_seconds = window_seconds
    
    def is_allowed(self, client_id: str) -> Tuple[bool, dict]:
        """
        Check if request is allowed under rate limit.
        Returns (allowed, info_dict)
        """
        now = time.time()
        window_start = now - self.window_seconds
        key = f"ratelimit:{client_id}"
        
        # Get current window data
        window_data = self.store.get(key) or []
        
        # Remove old timestamps outside window
        window_data = [ts for ts in window_data if ts > window_start]
        
        current_count = len(window_data)
        allowed = current_count < self.max_requests
        
        if allowed:
            window_data.append(now)
            self.store.set(key, window_data, self.window_seconds)
        
        return allowed, {
            "limit": self.max_requests,
            "remaining": max(0, self.max_requests - current_count - (1 if allowed else 0)),
            "reset": int(window_start + self.window_seconds)
        }


# Demo Rate Limiter
store = SimpleKeyValueStore()
limiter = SlidingWindowRateLimiter(store, max_requests=5, window_seconds=10)

print("=== Rate Limiter Demo (5 requests per 10 seconds) ===")

for i in range(7):
    allowed, info = limiter.is_allowed("client:123")
    status = "✅ Allowed" if allowed else "❌ Rate Limited"
    print(f"Request {i+1}: {status} | Remaining: {info['remaining']}/{info['limit']}")

## 9. Key Takeaways

### Key-Value Store Fundamentals
- 🔑 **Simple model**: Key → Value with O(1) operations
- ⏱️ **TTL support**: Automatic expiration for cache invalidation
- 🚀 **Use cases**: Caching, sessions, counters, queues

### Redis Patterns
- **Strings**: Counters, simple values, binary data
- **Lists**: Queues (LPUSH/RPOP), stacks (LPUSH/LPOP)
- **Sets**: Tags, unique collections, set operations
- **Hashes**: Object storage, field-level updates
- **Sorted Sets**: Leaderboards, priority queues, time-series

### DynamoDB Patterns
- **Single-table design**: Multiple entity types in one table
- **PK + SK**: Partition key for distribution, sort key for querying
- **GSI**: Alternative access patterns without table scans
- **Key design**: `ENTITY#id` pattern for multi-entity tables

### Caching Strategies
| Pattern | Write Latency | Read Latency | Consistency |
|---------|--------------|--------------|-------------|
| Cache-Aside | Low | Miss: High, Hit: Low | Eventual |
| Write-Through | High | Low | Strong |
| Write-Behind | Low | Low | Eventual |

### Best Practices
1. **Design keys carefully** - Include entity type and version
2. **Set appropriate TTLs** - Balance freshness vs. cache efficiency
3. **Monitor hit rates** - Target 95%+ for effective caching
4. **Plan for failure** - Cache should be optional, not required
5. **Use the right tool** - Redis for speed, DynamoDB for durability